# Exercise 1


In [7]:
from numpy import inf
from typing import Tuple, Optional

## Part (a) 
 Consider the following dynamical system
$$
x_{n+1} = 
\begin{cases}
-x_n + 1 + u_n & \text{if } -2 \leq -x_n + 1 + u_n \leq 2 \\
2 & \text{if } -x_n + 1 + u_n > 2 \\
-2 & \text{else}
\end{cases} \\
\forall \quad x_n \in \{-2, -1, 0, 1, 2\} \quad \text{and} \quad u_n \in \{-1, 0, 1\}
$$
And the cost function 
$$J = \left( \sum_{k=0}^{2} 2|x_k| + |u_k| \right) + x_3^2 \tag{1}$$
Use the dynamic programming algorithm to solve the finite horizon optimal control problem that minimizes $J$ . Show the different steps of the algorithms and present the results in a table including the cost to go and the optimal control at every stage.

### Solution 

First, let's define the possible values for $x$ & $u$. Then let us create functions for the system model and the current cost, as shown below.

In [8]:
# Possible values for state x and control u
X_POSS = [-2, -1, 0, 1, 2]
U_POSS = [-1, 0, 1]


def system_model(x_n: int, u_n: int) -> int:
    """
    Computes the next state of the system based on the current state and control.

    Args:
        x_n (int): The current state value, must be in X_POSS.
        u_n (int): The control input applied at the current step, must be in U_POSS.

    Raises:
        ValueError: If x_n is not a valid state in X_POSS.
        ValueError: If u_n is not a valid control in U_POSS.

    Returns:
        int: The next state, constrained to the interval [-2, 2].

    Description:
        - Calculates the next state `x_{n+1}` based on the system dynamics:
          x_{n+1} = -x_n + 1 + u_n if it is in [-2, 2].
        - If this computed value is greater than 2, returns 2.
        - If it is less than -2, returns -2.
    """
    # Validate current state
    if x_n not in X_POSS:
        raise ValueError(
            f"Invalid value for x: {x_n}. Must be one of {X_POSS}"
        )

    # Validate control input
    if u_n not in U_POSS:
        raise ValueError(
            f"Invalid value for u: {u_n}. Must be one of {U_POSS}"
        )

    # Calculate the potential next state based on system dynamics
    condition = -x_n + 1 + u_n

    # Constrain the next state to the interval [-2, 2]
    if -2 <= condition <= 2:
        return condition
    elif condition > 2:
        return 2
    else:
        return -2


def cost_func(x: int, u: int) -> int:
    """
    Calculates the cost based on the current state and control input.

    Args:
        x (int): The current state value.
        u (int): The control input applied at the current step.

    Returns:
        int: The computed cost, calculated as (2 * |x|) + |u|.
    """
    # Compute the cost using the given formula
    return (2 * abs(x)) + abs(u)

Next, we shall create a function to recursively compute the cost to go for each state at each stage and store it in a nested dictionary. Using this function, we shall compute the costs and optimal control which will be displayed in a table for easy reading.

In [9]:
# Memoization dictionary to store computed costs and optimal controls
memo = {}


def cost_to_go(stage: int, x: int) -> Tuple[float, Optional[int]]:
    """
    Computes the minimum cost to reach the final stage from the given stage and state,
    along with the optimal control input.

    Args:
        stage (int): The current stage (time step) in the process.
        x (int): The current state value.

    Returns:
        Tuple[float, Optional[int]]: A tuple containing the minimum cost (`j_min`) and
            the optimal control input (`u_opt`) for the given stage and state.

    Raises:
        None

    Description:
        - Utilizes memoization to store and retrieve previously computed costs and controls.
        - If the cost for the given stage and state is already computed, it returns the stored values.
        - If the current stage is the final stage (stage == 3), computes the cost as `x**2`
          and sets the optimal control to `None`.
        - Otherwise, iterates through all possible control inputs (`U_POSS`), computes the
          cost for each, and selects the control input that minimizes the total cost.
        - Stores the computed minimum cost and optimal control in the memoization dictionary.
    """
    # Check if the cost for the current stage and state is already memoized
    if stage in memo and x in memo[stage]:
        return memo[stage][x]['j'], memo[stage][x]['u']
    else:
        # Initialize the dictionary for the current stage if not present
        memo.setdefault(stage, {})

    if stage == 3:
        # Base case: final stage, cost is x squared, no control input
        j_min = x ** 2
        u_opt = None
        memo[stage][x] = {'u': u_opt, 'j': j_min}
        return j_min, u_opt

    # Initialize minimum cost to infinity and no optimal control
    j_min = inf
    u_opt = None

    # Iterate through all possible control inputs to find the optimal one
    for u_i in U_POSS:
        # Compute the next state based on current state and control input
        next_x = system_model(x_n=x, u_n=u_i)
        # Compute the total cost: current cost + cost to go from the next state
        j_i = cost_func(x=x, u=u_i) + cost_to_go(stage=stage + 1, x=next_x)[0]
        # Update the minimum cost and optimal control if a lower cost is found
        if j_i < j_min:
            j_min = j_i
            u_opt = u_i

    # Memoize the computed minimum cost and optimal control for the current stage and state
    memo[stage][x] = {'u': u_opt, 'j': j_min}
    return j_min, u_opt


# Define the header with columns for each stage
header = ["State"]
for stage in range(4):
    header.append(f"J_{stage}")
    header.append(f"u_{stage}")


# Print the header with formatted spacing
print("{:<6} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format(*header))

# Iterate through each possible state and compute costs and optimal controls
for x in X_POSS:
    row = [x]
    for stage in range(4):
        j, u = cost_to_go(stage=stage, x=x)
        row.append(j)
        row.append(u)

    # Format and print the row with aligned columns
    print("{:<6} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format(*row))

State  J_0        u_0        J_1        u_1        J_2        u_2        J_3       
-2     10         0          9          0          8          0          4         
-1     6          -1         5          -1         4          -1         1         
0      3          -1         2          -1         1          -1         0         
1      4          0          3          0          2          0          1         
2      7          1          6          1          5          0          4         


## Part (b)
What is the sequence of control actions, states and the optimal cost if $ x_0 = 0 $, if $ x_0 = -2 $ and if $ x_0 = 2 $. 

### Solution is given in the report, computational code given below -

In [10]:
x = 2  # Change this value for a different starting state
for i in range(0, 3):
    u = memo[i][x]['u']
    j = memo[i][x]['j']
    xp1 = system_model(x_n=x, u_n=u)
    print(f"For x = {x}, u = {u} so x+ = {xp1} with J = {j}")
    x = xp1

print(f"For x = {x} with J = {memo[i+1][x]['j']}")

For x = 2, u = 1 so x+ = 0 with J = 7
For x = 0, u = -1 so x+ = 0 with J = 2
For x = 0, u = -1 so x+ = 0 with J = 1
For x = 0 with J = 0


## Part (c)

Assume now that the constant term $ 1 $ in the previous dynamics can sometimes be $ 0 $ with probability $ 0.4 $. We can now write the dynamics as
$$
x_{n+1} = 
\begin{cases}
-x_n + \omega_n + u_n & \text{if } -2 \leq -x_n + \omega_n + u_n \leq 2 \\
2 & \text{if } -x_n + \omega_n + u_n > 2 \\
-2 & \text{else}
\end{cases}
$$
where $ x_n \in \{-2, -1, 0, 1, 2\} $, $ u_n \in \{-1, 0, 1\} $ and $ \omega_n \in \{0, 1\} $ is a random variable with probability distribution $ p(\omega_n = 0) = 0.4, p(\omega_n = 1) = 0.6 $. The cost function to minimize becomes
$$
J = \mathbb{E} \left( \sum_{k=0}^{2} 2|x_k| + |u_k| \right) + x_3^2 \tag{2}
$$
Use the dynamic programming algorithm to solve the finite horizon optimal control problem that minimizes $ J $. Show the different steps of the algorithms and present the results in a table including the cost to go and the optimal control at every stage.

### Solution 

First, let's define the possible values for $\omega$. Then let us create functions for the system model to include $\omega$, as shown below. The cost remains the same as before as it is purely deterministic.

In [11]:
# Creating a list to store the possible values of omega and respective probabilites
OMEGA_POSS = [0, 1]
PROB_OMEGA_POSS = [0.4, 0.6]


def system_model_prob(x_n: int, u_n: int, omega_n: int) -> int:
    """
    Computes the next state of the system based on the current state, control input, and random variable.

    The system dynamics are defined as:
        x_{n+1} = 
            -x_n + ω_n + u_n,  if -2 ≤ -x_n + ω_n + u_n ≤ 2
            2,                  if -x_n + ω_n + u_n > 2
            -2,                 otherwise

    Args:
        x_n (int): The current state. Must be one of {-2, -1, 0, 1, 2}.
        u_n (int): The control input. Must be one of {-1, 0, 1}.
        omega_n (int): The random variable. Must be either 0 or 1.
                       ω_n = 0 with probability 0.4 and ω_n = 1 with probability 0.6.

    Raises:
        ValueError: If `x_n` is not in {-2, -1, 0, 1, 2}.
        ValueError: If `u_n` is not in {-1, 0, 1}.
        ValueError: If `omega_n` is not in {0, 1}.

    Returns:
        int: The next state `x_{n+1}` after applying the system dynamics.
    """
    # Validate current state
    if x_n not in X_POSS:
        raise ValueError(
            f"Invalid value for x: {x_n}. Must be one of {X_POSS}"
        )

    # Validate control input
    if u_n not in U_POSS:
        raise ValueError(
            f"Invalid value for u: {u_n}. Must be one of {U_POSS}"
        )

    # Validate random variable input
    if omega_n not in OMEGA_POSS:
        raise ValueError(
            f"Invalid value for omega: {omega_n}. Must be one of {OMEGA_POSS}"
        )

    # Compute the condition based on system dynamics
    condition = -x_n + omega_n + u_n

    # Determine the next state based on the condition
    if -2 <= condition <= 2:
        return condition
    elif condition > 2:
        return 2
    else:
        return -2

Next, we shall create a function to recursively compute the cost to go for each state at each stage and store it in a nested dictionary. Using this function, we shall compute the costs and optimal control which will be displayed in a table for easy reading.

In [12]:
# Memoization dictionary to store computed costs and optimal controls
memo_prob = {}


def cost_to_go_prob(stage: int, x: int) -> Tuple[float, Optional[int]]:
    """
    Computes the minimum expected cost and the optimal control input for a given stage and state using dynamic programming.

    This function utilizes memoization to store and retrieve previously computed costs to improve efficiency.

    The cost function to minimize is defined as:
        J = E[ Σ (2|x_k| + |u_k|) ] + x_3^2
        where the expectation is taken over the random variable ω with p(ω=0) = 0.4 and p(ω=1) = 0.6.

    Args:
        stage (int): The current stage (time step) in the finite horizon problem. Must be between 0 and 3 inclusive.
        x (int): The current state. Must be one of {-2, -1, 0, 1, 2}.

    Raises:
        ValueError: If the stage is outside the valid range or if the state is invalid.

    Returns:
        Tuple[float, Optional[int]]: A tuple containing:
            - The minimum expected cost (`j_min`) from the current stage and state.
            - The optimal control input (`u_opt`) that achieves this minimum cost. Returns `None` if no control input is applicable.
    """
    # Check if the cost for the current stage and state is already memoized
    if stage in memo_prob and x in memo_prob[stage]:
        return memo_prob[stage][x]['j'], memo_prob[stage][x]['u']
    else:
        # Initialize the dictionary for the current stage if not present
        memo_prob.setdefault(stage, {})

    if stage == 3:
        # Base case: final stage, cost is x squared, no control input
        j_min = x ** 2
        u_opt = None
        memo_prob[stage][x] = {'u': u_opt, 'j': j_min}
        return j_min, u_opt

    # Initialize minimum cost to infinity and no optimal control
    j_min = inf
    u_opt = None

    # Iterate through all possible control inputs to find the optimal one
    for u_i in U_POSS:
        # Compute the next state based on current state, control input, and omega = 0
        next_x_0 = system_model_prob(x_n=x, u_n=u_i, omega_n=OMEGA_POSS[0])
        # Compute the next state based on current state, control input, and omega = 1
        next_x_1 = system_model_prob(x_n=x, u_n=u_i, omega_n=OMEGA_POSS[1])
        # Compute the total expected cost:
        # Current cost + weighted cost to go from the next states
        j_i = (
            cost_func(x=x, u=u_i) +
            (PROB_OMEGA_POSS[0] * cost_to_go_prob(stage=stage + 1, x=next_x_0)[0]) +
            (PROB_OMEGA_POSS[1] *
             cost_to_go_prob(stage=stage + 1, x=next_x_1)[0])
        )
        # Update the minimum cost and optimal control if a lower cost is found
        if j_i < j_min:
            j_min = j_i
            u_opt = u_i

    # Memoize the computed minimum cost and optimal control for the current stage and state
    memo_prob[stage][x] = {'u': u_opt, 'j': j_min}
    return j_min, u_opt


# Define the header with columns for each stage
header = ["State"]
for stage in range(4):
    header.append(f"J_{stage}")
    header.append(f"u_{stage}")

# Print the header with formatted spacing
print("{:<6} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format(*header))

# Iterate through each possible state and compute costs and optimal controls
for x in X_POSS:
    row = [x]
    for stage in range(4):
        j, u = cost_to_go_prob(stage=stage, x=x)
        row.append(round(j, 3))
        row.append(u)

    # Format and print the row with aligned columns
    print("{:<6} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format(*row))

State  J_0        u_0        J_1        u_1        J_2        u_2        J_3       
-2     10.6       -1         9.2        -1         7.8        -1         4         
-1     5.952      -1         4.68       -1         3.6        -1         1         
0      2.952      0          1.68       0          0.6        0          0         
1      4.88       0          3.8        0          2.4        0          1         
2      7.88       1          6.8        1          5.4        1          4         


## Part (d)
Compare the costs and optimal control from the deterministic and probabilistic models and explain where, in your opinion, the differences come from.

### Solution is given in the report